In [1]:
from __future__ import print_function, division
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} requests
import requests
#responsewx = requests.get("https://api.openweathermap.org/data/2.5/find?lat=39.4&lon=-77.3&cnt=5&units=imperial&appid=edc8e66602652591b418f7492de29f82")
#responsewx = requests.get("https://api.openweathermap.org/data/2.5/weather?zip=21774,us&units=imperial&appid=edc8e66602652591b418f7492de29f82")
responsewx = requests.get("https://api.openweathermap.org/data/2.5/box/city?bbox=-77.5,39.2,-77,39.7,10&units=imperial&appid=edc8e66602652591b418f7492de29f82")
## Let's check the status code of the response
#print(responsewx.json())
print(responsewx.content)

This program is blocked by group policy. For more information, contact your system administrator.


{"cod":200,"calctime":0.001911113,"cnt":2,"list":[{"id":4355585,"dt":1601673239,"name":"Frederick","coord":{"Lon":-77.41,"Lat":39.41},"main":{"temp":62.94,"feels_like":53.85,"temp_min":61,"temp_max":66.2,"pressure":1018,"humidity":39},"visibility":10000,"wind":{"speed":11.40837508947745,"deg":310},"rain":null,"snow":null,"clouds":{"today":40},"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03d"}]},{"id":4352728,"dt":1601673238,"name":"Damascus","coord":{"Lon":-77.2,"Lat":39.29},"main":{"temp":62.76,"feels_like":55.94,"temp_min":61,"temp_max":64.99,"pressure":1018,"humidity":48},"visibility":10000,"wind":{"speed":9.171438797423049,"deg":300},"rain":null,"snow":null,"clouds":{"today":75},"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04d"}]}]}


In [2]:
import pandas as pd 
import os, ssl

if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context
    
data = pd.read_json(r'https://api.openweathermap.org/data/2.5/box/city?bbox=-77.5,39.2,-77,39.7,10&units=imperial&appid=edc8e66602652591b418f7492de29f82')
#data = pd.read_json(responsewx.content) 
print(data) 

   calctime  cnt  cod                                               list
0  0.001911    2  200  {u'clouds': {u'today': 40}, u'name': u'Frederi...
1  0.001911    2  200  {u'clouds': {u'today': 75}, u'name': u'Damascu...


In [3]:
from pandas.io.json import json_normalize  
 

# lets put the data into a pandas df 
# clicking on raw_nyc_phil.json under "Input Files" 
# tells us parent node is 'programs' 
wxobs = json_normalize(data['list']) 
print(wxobs)

   clouds.today  coord.Lat  coord.Lon          dt       id  main.feels_like  \
0            40      39.41     -77.41  1601673239  4355585            53.85   
1            75      39.29     -77.20  1601673238  4352728            55.94   

   main.humidity  main.pressure  main.temp  main.temp_max  main.temp_min  \
0             39           1018      62.94          66.20             61   
1             48           1018      62.76          64.99             61   

        name  rain  snow  visibility  \
0  Frederick  None  None       10000   
1   Damascus  None  None       10000   

                                             weather  wind.deg  wind.speed  
0  [{u'description': u'scattered clouds', u'main'...       310   11.408375  
1  [{u'description': u'broken clouds', u'main': u...       300    9.171439  


In [4]:
def compthetae(temp,RH,pressure):
    tempc = (temp - 32) / 1.8
    print("Temp_C = ", tempc)
    thetae = (273.15 + tempc)*((1000/pressure)**0.286)+(3 * (RH * (3.884266 * 10**
         ((7.5 * tempc)/(237.7 + tempc)))/100))
    return thetae

In [5]:
wx_data = json_normalize(data.to_dict('series'), 'list') 
print(wx_data)
loc1 = wx_data['name'][0]
windspd_loc1 = wx_data['wind'][0]['speed']
winddir_loc1 = wx_data['wind'][0]['deg']
temp_loc1 = wx_data['main'][0]['temp']
RH_loc1 = wx_data['main'][0]['humidity']
pressure_loc1 = wx_data['main'][0]['pressure']
thetae_loc1 = compthetae(temp_loc1,RH_loc1,pressure_loc1)
print("Location: ", loc1)
print("Wind speed: ", windspd_loc1)
print("Wind speed: ", winddir_loc1)
print("Temperature: ", temp_loc1)
print("Humidity: ", RH_loc1)
print("Pressure: ", pressure_loc1)
print("Theta-e: ", thetae_loc1)

loc2 = wx_data['name'][1]
windspd_loc2 = wx_data['wind'][1]['speed']
winddir_loc2 = wx_data['wind'][1]['deg']
temp_loc2 = wx_data['main'][1]['temp']
RH_loc2 = wx_data['main'][1]['humidity']
pressure_loc2 = wx_data['main'][1]['pressure']
thetae_loc2 = compthetae(temp_loc2,RH_loc2,pressure_loc2)
print("Location: ", loc2)
print("Wind speed: ", windspd_loc2)
print("Wind speed: ", winddir_loc2)
print("Temperature: ", temp_loc2)
print("Humidity: ", RH_loc2)
print("Pressure: ", pressure_loc2)
print("Theta-e: ", thetae_loc2)

delta_thetae = thetae_loc2 - thetae_loc1
print("Theta-e difference between location 1 and location 2 = ", delta_thetae)

if winddir_loc2 < winddir_loc1:
    winddirdiff = winddir_loc1 - winddir_loc2
    if winddirdiff >= 45:
        wshift = True
    else:
        wshift = False
        
    if delta_thetae > 0:
        if thetae_loc2 > 330:
            svrstorms = True
        elif thetae_loc2 > 310:
            svrstorms = False
            storms = True
        else:
            svrstorms = False
            storms = False
            cumulus = True
    elif thetae_loc2 > 300:
        svrstorms = False
        storms = False
        cumulus = True
    else:
        svrstorms = False
        storms = False
        cumulus = False
        
    div = False
    
    print(wshift,div,svrstorms,storms,cumulus)
    
    if wshift == True:
        print("Wind convergence - frontal boundary present")
    else:
        print("Wind convergence - frontal boundary not present")
    if svrstorms == True:
        print("Severe storm and MCS development possible")
    elif storms == True:
        print("Scattered storm development possible")
    else:        
        print("Cumulus cloud development possible")
    
elif winddir_loc2 > winddir_loc1:
    whsift = False
    div = True
    print(whsift,div)
    print("Wind divergence - frontal boundary not present")
else:
    whsift = False
    div = True
    print(whsift,div)
    print("Frontal boundary not present")
 

           clouds                            coord          dt       id  \
0  {u'today': 40}  {u'Lat': 39.41, u'Lon': -77.41}  1601673239  4355585   
1  {u'today': 75}   {u'Lat': 39.29, u'Lon': -77.2}  1601673238  4352728   

                                                main       name  rain  snow  \
0  {u'temp': 62.94, u'temp_max': 66.2, u'humidity...  Frederick  None  None   
1  {u'temp': 62.76, u'temp_max': 64.99, u'humidit...   Damascus  None  None   

   visibility                                            weather  \
0       10000  [{u'main': u'Clouds', u'id': 802, u'icon': u'0...   
1       10000  [{u'main': u'Clouds', u'id': 803, u'icon': u'0...   

                                     wind  
0  {u'speed': 11.4083750895, u'deg': 310}  
1  {u'speed': 9.17143879742, u'deg': 300}  
Temp_C =  17.1888888889
Location:  Frederick
Wind speed:  11.4083750895
Wind speed:  310
Temperature:  62.94
Humidity:  39
Pressure:  1018
Theta-e:  303.424957458
Temp_C =  17.0888888889
Location:  D